In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  


import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization", "other"]

rows = []

In [3]:
# 3 classes

NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (200, 20, 1)

norm=True

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[134.1855, 122.7346, 118.3749], std=[70.5125, 64.4848, 66.5604])])
else:
    transform=None

    
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  

test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=3)

weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 3 class normal

learning_rates = [1e-2, 5e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochsf):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [20]:
#  3 class ratio

learning_rates =  [1e-2, 1e-3, 5e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


Test set: Avg. loss: 0.0011114914417266846, AUC: 0.36460475000000003


Test set: Avg. loss: 0.0023845341205596922, AUC: 0.6524673333333334


Test set: Avg. loss: 0.0022546345392862955, AUC: 0.6776095833333334


Test set: Avg. loss: 0.0021248392264048258, AUC: 0.6789875833333333


Test set: Avg. loss: 0.0010890957514444986, AUC: 0.6553961666666667


Test set: Avg. loss: 0.0026082831223805747, AUC: 0.6564603333333334


Test set: Avg. loss: 0.0025372488498687745, AUC: 0.6645089999999999


Test set: Avg. loss: 0.002328646183013916, AUC: 0.6728794166666666


Test set: Avg. loss: 0.0011088247299194335, AUC: 0.3817441666666667


Test set: Avg. loss: 0.002548003117243449, AUC: 0.6628840833333333


Test set: Avg. loss: 0.002347040812174479, AUC: 0.6801826666666666


Test set: Avg. loss: 0.0024017000993092853, AUC: 0.6815919166666666


Test set: Avg. loss: 0.0011053735812505087, AUC: 0.47676008333333336


Test set: Avg. loss: 0.0021743202209472657, AUC: 0.6694651666666668


Test set: Avg. loss:

In [21]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [12]:
# 3 class oversampled 

learning_rates = [5e-2, 1e-2, 1e-3, 5e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0011114914417266846, AUC: 0.36460475000000003


Test set: Avg. loss: 0.004324390570322673, AUC: 0.7364573333333334


Test set: Avg. loss: 0.0037501402695973712, AUC: 0.7574884166666666


Test set: Avg. loss: 0.004474962711334228, AUC: 0.79239925


Test set: Avg. loss: 0.0011069943904876709, AUC: 0.37739375


Test set: Avg. loss: 0.0030027631918589275, AUC: 0.7237471666666666


Test set: Avg. loss: 0.005124391555786133, AUC: 0.7233443333333334


Test set: Avg. loss: 0.005480806827545166, AUC: 0.7494814999999999


Test set: Avg. loss: 0.0011040428479512532, AUC: 0.5238316666666667


Test set: Avg. loss: 0.0026677467823028565, AUC: 0.7431178333333333


Test set: Avg. loss: 0.0030789877573649087, AUC: 0.7665563333333333


Test set: Avg. loss: 0.003713517268498739, AUC: 0.7753674999999999


Test set: Avg. loss: 0.0011082092126210531, AUC: 0.41351783333333336


Test set: Avg. loss: 0.0037240614891052245, AUC: 0.7545196666666666


Test set: Avg. loss: 0.004394479274749


Test set: Avg. loss: 0.0010080868005752563, AUC: 0.7724891666666668


Test set: Avg. loss: 0.0017206243673960368, AUC: 0.7600154166666666


Test set: Avg. loss: 0.0025581661065419516, AUC: 0.7579385


Test set: Avg. loss: 0.001097543716430664, AUC: 0.5109705


Test set: Avg. loss: 0.0011088798046112061, AUC: 0.7401753333333335


Test set: Avg. loss: 0.0018518801132837931, AUC: 0.7338064166666666


Test set: Avg. loss: 0.002743795871734619, AUC: 0.731045


Test set: Avg. loss: 0.0011036570072174073, AUC: 0.46421399999999996


Test set: Avg. loss: 0.0008840240041414896, AUC: 0.7970324999999999


Test set: Avg. loss: 0.0013941014210383097, AUC: 0.7866110000000001


Test set: Avg. loss: 0.0019833201964696246, AUC: 0.7845465000000001


Test set: Avg. loss: 0.0011008837223052978, AUC: 0.4977171666666667


Test set: Avg. loss: 0.0009412099123001098, AUC: 0.7853393333333334


Test set: Avg. loss: 0.0015829279820124308, AUC: 0.7723570000000001


Test set: Avg. loss: 0.0021948086420694986, AUC:

In [13]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [14]:
#  3 class SMOTE

learning_rates = [5e-2, 1e-2, 1e-3, 5e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0011012120644251506, AUC: 0.5157660833333333


Test set: Avg. loss: 0.001253303090731303, AUC: 0.8061060000000001


Test set: Avg. loss: 0.00180307137966156, AUC: 0.7786304166666667


Test set: Avg. loss: 0.0020693496068318685, AUC: 0.7883991666666667


Test set: Avg. loss: 0.0011086985270182292, AUC: 0.44565125000000005


Test set: Avg. loss: 0.0015388530095418295, AUC: 0.7731013333333333


Test set: Avg. loss: 0.001989365021387736, AUC: 0.7840317499999999


Test set: Avg. loss: 0.0023551328976949056, AUC: 0.7794379166666667


Test set: Avg. loss: 0.0010967851877212525, AUC: 0.5745733333333333


Test set: Avg. loss: 0.001639639377593994, AUC: 0.7576433333333333


Test set: Avg. loss: 0.0017591052452723185, AUC: 0.7736256666666668


Test set: Avg. loss: 0.0023112374146779377, AUC: 0.7581950833333334


Test set: Avg. loss: 0.0011002494891484578, AUC: 0.4803038333333333


Test set: Avg. loss: 0.0014218109846115112, AUC: 0.7767816666666668


Test set: Avg. loss: 0.


Test set: Avg. loss: 0.0009895188609759012, AUC: 0.7346210000000001


Test set: Avg. loss: 0.0011809653441111248, AUC: 0.7438486666666666


Test set: Avg. loss: 0.0013460488319396973, AUC: 0.7523574166666666


Test set: Avg. loss: 0.0011083951393763224, AUC: 0.42329483333333334


Test set: Avg. loss: 0.0009447627067565918, AUC: 0.7579581666666666


Test set: Avg. loss: 0.0009011501669883728, AUC: 0.7721625833333333


Test set: Avg. loss: 0.0009771966139475504, AUC: 0.8049443333333333


Test set: Avg. loss: 0.0010899279514948526, AUC: 0.6394894166666667


Test set: Avg. loss: 0.0009647173484166464, AUC: 0.7354416666666667


Test set: Avg. loss: 0.0010964591900507609, AUC: 0.7503926666666666


Test set: Avg. loss: 0.0012541187206904093, AUC: 0.7699424166666667


Test set: Avg. loss: 0.0011026158332824707, AUC: 0.47225925


Test set: Avg. loss: 0.0010245993137359619, AUC: 0.7351810833333333


Test set: Avg. loss: 0.0010234986543655395, AUC: 0.7520389999999999


Test set: Avg. loss: 0.001

In [15]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [14]:
# 3 class capped smote 


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, None] 
        rows.append(row)

print(rows)


Test set: Avg. loss: 0.0011079707543055217, AUC: 0.4224414166666666


Test set: Avg. loss: 0.0024819238980611164, AUC: 0.6706555833333333


Test set: Avg. loss: 0.002305363098780314, AUC: 0.6838636666666668


Test set: Avg. loss: 0.002249549627304077, AUC: 0.6926545000000001


Test set: Avg. loss: 0.0011115634441375732, AUC: 0.44383383333333337


Test set: Avg. loss: 0.0018068215847015382, AUC: 0.7456852500000001


Test set: Avg. loss: 0.0023212509950002033, AUC: 0.7392154999999999


Test set: Avg. loss: 0.0015493378241856893, AUC: 0.8035191666666667


Test set: Avg. loss: 0.0010960235595703124, AUC: 0.5768070833333333


Test set: Avg. loss: 0.0017772521575291952, AUC: 0.7280496666666667


Test set: Avg. loss: 0.002030446648597717, AUC: 0.7317598333333333


Test set: Avg. loss: 0.002225544532140096, AUC: 0.7191306666666666


Test set: Avg. loss: 0.0011004793246587117, AUC: 0.5102983333333333


Test set: Avg. loss: 0.002655739943186442, AUC: 0.6715455


Test set: Avg. loss: 0.002424101


Test set: Avg. loss: 0.0008978073398272197, AUC: 0.7981495000000001


Test set: Avg. loss: 0.0010476662516593932, AUC: 0.797174


Test set: Avg. loss: 0.0013957345088322958, AUC: 0.801761


Test set: Avg. loss: 0.0011011792421340943, AUC: 0.5008129166666667


Test set: Avg. loss: 0.0010919816493988037, AUC: 0.7615899166666665


Test set: Avg. loss: 0.0017090630531311035, AUC: 0.75741025


Test set: Avg. loss: 0.0017677673896153768, AUC: 0.7701735833333333


Test set: Avg. loss: 0.0011115765571594238, AUC: 0.4346426666666667


Test set: Avg. loss: 0.0011828872362772624, AUC: 0.7415978333333332


Test set: Avg. loss: 0.0012650069793065388, AUC: 0.7911315833333333


Test set: Avg. loss: 0.0015466095209121704, AUC: 0.7957371666666666


Test set: Avg. loss: 0.001096881588300069, AUC: 0.5596155833333333


Test set: Avg. loss: 0.0011262359221776326, AUC: 0.7633


Test set: Avg. loss: 0.0018325774272282918, AUC: 0.7689121666666665


Test set: Avg. loss: 0.0017438907623291016, AUC: 0.797137833


Test set: Avg. loss: 0.0010901000102361044, AUC: 0.7558843333333334


Test set: Avg. loss: 0.001592508633931478, AUC: 0.7648861666666665


Test set: Avg. loss: 0.0015164965391159058, AUC: 0.793024


Test set: Avg. loss: 0.0011064399878184, AUC: 0.44293325


Test set: Avg. loss: 0.0011655902862548828, AUC: 0.7469121666666666


Test set: Avg. loss: 0.001499106486638387, AUC: 0.7580291666666666


Test set: Avg. loss: 0.0018441102107365926, AUC: 0.7539993333333332


Test set: Avg. loss: 0.0011045443614323934, AUC: 0.46125925000000007


Test set: Avg. loss: 0.0011498111089070639, AUC: 0.7564623333333333


Test set: Avg. loss: 0.0015798531770706177, AUC: 0.7602367499999999


Test set: Avg. loss: 0.002093439976374308, AUC: 0.7589875833333334


Test set: Avg. loss: 0.001103798508644104, AUC: 0.4563561666666667


Test set: Avg. loss: 0.0010767038265864055, AUC: 0.7555844999999999


Test set: Avg. loss: 0.0013137869834899901, AUC: 0.7909311666666666


Test set: Avg. loss: 0.0018448962370554607,

In [15]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [16]:
# 3 class euclidean distance capped smote 


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'euclidean'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0011004855235417683, AUC: 0.4887625


Test set: Avg. loss: 0.0011042307217915854, AUC: 0.7546737499999999


Test set: Avg. loss: 0.001469256083170573, AUC: 0.7447474166666667


Test set: Avg. loss: 0.0018616555134455363, AUC: 0.7519853333333333


Test set: Avg. loss: 0.0011009955008824666, AUC: 0.4859675833333334


Test set: Avg. loss: 0.0014954797426859538, AUC: 0.7307878333333333


Test set: Avg. loss: 0.0014571794668833415, AUC: 0.7434764166666666


Test set: Avg. loss: 0.001918233076731364, AUC: 0.7510033333333332


Test set: Avg. loss: 0.0010990285078684488, AUC: 0.5017333333333333


Test set: Avg. loss: 0.0015441822210947673, AUC: 0.7289679166666666


Test set: Avg. loss: 0.0013621874650319417, AUC: 0.7503736666666666


Test set: Avg. loss: 0.0020592970053354897, AUC: 0.7446693333333333


Test set: Avg. loss: 0.001104023019472758, AUC: 0.4327043333333333


Test set: Avg. loss: 0.0011127008597056072, AUC: 0.7686645


Test set: Avg. loss: 0.00162692983945210


Test set: Avg. loss: 0.00109506889184316, AUC: 0.6075227500000001


Test set: Avg. loss: 0.0010930527051289877, AUC: 0.7969098333333333


Test set: Avg. loss: 0.00121285879611969, AUC: 0.7962791666666668


Test set: Avg. loss: 0.0015341360569000244, AUC: 0.7968086666666668


Test set: Avg. loss: 0.00110688320795695, AUC: 0.39512858333333334


Test set: Avg. loss: 0.0010997189680735271, AUC: 0.7778755


Test set: Avg. loss: 0.0013350268205006917, AUC: 0.77851625


Test set: Avg. loss: 0.0011274513800938924, AUC: 0.81341375


Test set: Avg. loss: 0.0011012434562047322, AUC: 0.49900916666666667


Test set: Avg. loss: 0.001257138729095459, AUC: 0.7524218333333333


Test set: Avg. loss: 0.002043050686518351, AUC: 0.7320953333333332


Test set: Avg. loss: 0.0026825573444366453, AUC: 0.7337874166666666


Test set: Avg. loss: 0.0010923410654067993, AUC: 0.6525029999999999


Test set: Avg. loss: 0.0009580798546473185, AUC: 0.7926976666666667


Test set: Avg. loss: 0.0015643794139226278, AUC: 0


Test set: Avg. loss: 0.0010996352036794026, AUC: 0.7698611666666667


Test set: Avg. loss: 0.0018259684642155966, AUC: 0.7553820833333332


Test set: Avg. loss: 0.001500830332438151, AUC: 0.7931090833333333


Test set: Avg. loss: 0.00109957222143809, AUC: 0.5198979166666666


Test set: Avg. loss: 0.001127657175064087, AUC: 0.75847275


Test set: Avg. loss: 0.0012819421291351317, AUC: 0.7650260000000001


Test set: Avg. loss: 0.0013352120717366536, AUC: 0.7699018333333333


Test set: Avg. loss: 0.0011067168315251669, AUC: 0.44954541666666664


Test set: Avg. loss: 0.0012927870750427245, AUC: 0.7444374166666666


Test set: Avg. loss: 0.0011801127195358277, AUC: 0.7940649999999999


Test set: Avg. loss: 0.0015621389547983806, AUC: 0.7898746666666666


Test set: Avg. loss: 0.0011187699635823568, AUC: 0.4121849166666667


Test set: Avg. loss: 0.0011955602169036864, AUC: 0.770215


Test set: Avg. loss: 0.0012250461181004843, AUC: 0.802267


Test set: Avg. loss: 0.0017819425264994303, AUC: 0

In [17]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [18]:
# 3 class cosine distance capped smote 


momentum=0
learning_rates = [1e-2, 1e-3, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'cosine'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0011052825848261514, AUC: 0.43978183333333337


Test set: Avg. loss: 0.001242885112762451, AUC: 0.7556419999999999


Test set: Avg. loss: 0.0017786058187484741, AUC: 0.7451270833333333


Test set: Avg. loss: 0.001355748494466146, AUC: 0.7903454166666667


Test set: Avg. loss: 0.0010999015967051188, AUC: 0.5015735


Test set: Avg. loss: 0.001183523694674174, AUC: 0.7529743333333333


Test set: Avg. loss: 0.0017083600759506227, AUC: 0.7684294999999999


Test set: Avg. loss: 0.0020539085070292156, AUC: 0.7834125


Test set: Avg. loss: 0.0010926770766576132, AUC: 0.5972151666666666


Test set: Avg. loss: 0.0012481575806935629, AUC: 0.7396061666666668


Test set: Avg. loss: 0.001280996561050415, AUC: 0.774436


Test set: Avg. loss: 0.0013489604791005452, AUC: 0.8045683333333334


Test set: Avg. loss: 0.0011085643370946248, AUC: 0.41532358333333336


Test set: Avg. loss: 0.001295187473297119, AUC: 0.7403531666666666


Test set: Avg. loss: 0.0014481489260991414, AUC: 0


Test set: Avg. loss: 0.001102006713549296, AUC: 0.5132036666666666


Test set: Avg. loss: 0.0012708915074666342, AUC: 0.7639296666666667


Test set: Avg. loss: 0.0015375802119572957, AUC: 0.7768043333333333


Test set: Avg. loss: 0.002539357344309489, AUC: 0.7442037499999999


Test set: Avg. loss: 0.0011142092148462932, AUC: 0.3598644166666667


Test set: Avg. loss: 0.0010708965063095093, AUC: 0.77946075


Test set: Avg. loss: 0.001908510684967041, AUC: 0.7509295000000001


Test set: Avg. loss: 0.0016529592672983805, AUC: 0.79660725


Test set: Avg. loss: 0.0011188224951426187, AUC: 0.3916071666666667


Test set: Avg. loss: 0.0012840594053268432, AUC: 0.7236514166666667


Test set: Avg. loss: 0.0012736543416976929, AUC: 0.7729545833333334


Test set: Avg. loss: 0.002117136081059774, AUC: 0.7873815000000001


Test set: Avg. loss: 0.0011116609573364258, AUC: 0.357033


Test set: Avg. loss: 0.0010108190377553304, AUC: 0.7509784166666668


Test set: Avg. loss: 0.0012108296553293863, AUC: 


Test set: Avg. loss: 0.001682835817337036, AUC: 0.7700859166666666


Test set: Avg. loss: 0.0016216527223587036, AUC: 0.8116925


Test set: Avg. loss: 0.0011027799050013225, AUC: 0.46017141666666667


Test set: Avg. loss: 0.0009993523557980854, AUC: 0.7918486666666666


Test set: Avg. loss: 0.0013525794744491577, AUC: 0.7881920833333332


Test set: Avg. loss: 0.0015977139075597127, AUC: 0.8012561666666667


Test set: Avg. loss: 0.001100353757540385, AUC: 0.47812058333333335


Test set: Avg. loss: 0.0011181455055872599, AUC: 0.7524395833333334


Test set: Avg. loss: 0.0015768436988194783, AUC: 0.7707788333333333


Test set: Avg. loss: 0.001684269905090332, AUC: 0.8039378333333334


Test set: Avg. loss: 0.0010977291266123454, AUC: 0.5273618333333334


Test set: Avg. loss: 0.0012737759749094646, AUC: 0.7475231666666667


Test set: Avg. loss: 0.001591060439745585, AUC: 0.7671234166666666


Test set: Avg. loss: 0.0019627333879470824, AUC: 0.7713725


Test set: Avg. loss: 0.0011134448846181

In [19]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [8]:
# 3 class triplet loss capped smote


momentum=0
learning_rates = [1e-2, 5e-3, 1e-3]

cap_aucs = []

start_epoch = 2

loss_caps = [3] 
loss_fn_args = {}


for loss_cap in loss_caps:
    
    loss_fn_args['loss_cap'] = loss_cap
    
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['print_capped'] = False
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                loss_fn_args['print_capped'] = False
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, None]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.001108316699663798, AUC: 0.40338908333333334


Test set: Avg. loss: 0.0010837951103846231, AUC: 0.7395576666666667


Test set: Avg. loss: 0.0013000988165537517, AUC: 0.7524381666666667


Test set: Avg. loss: 0.0015997310082117716, AUC: 0.772925


Test set: Avg. loss: 0.0011064944664637248, AUC: 0.41710908333333335


Test set: Avg. loss: 0.0012484875917434693, AUC: 0.7461605000000001


Test set: Avg. loss: 0.0010611064831415812, AUC: 0.7906749999999999


Test set: Avg. loss: 0.0017651242812474568, AUC: 0.7832611666666667


Test set: Avg. loss: 0.0011017123063405355, AUC: 0.5216805833333333


Test set: Avg. loss: 0.0013332428534825642, AUC: 0.7182235


Test set: Avg. loss: 0.0014227912425994872, AUC: 0.7462981666666667


Test set: Avg. loss: 0.0014862947066624959, AUC: 0.7949613333333335


Test set: Avg. loss: 0.0011074897845586142, AUC: 0.4555526666666667


Test set: Avg. loss: 0.0013799664974212646, AUC: 0.722566


Test set: Avg. loss: 0.0015054897467295329, AUC

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 